This code uses the Python Open CV library to align red and rededge images in the format tif
and combines them to produce an aligned RGB image in the format JPG. In addition, 
this code transfers the GPS information from the image in the reference band to 
the aligned RGB image.

Extensions asociated to each name file refers to a diferente band as is detailed below. 

- '*_1.tif' = Blue Band
- '*_2.tif' = Green Band
- '*_3.tif' = Red Band
- '*_4.tif' = Near Infrarred Band
- '*_5.tif' = Rededge Band

The output can be used to compute the NDRE vegetation index.

In [1]:
##Preamble
%matplotlib inline  
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import traceback
import sys
from PIL import Image, ImageFilter
import PIL
import matplotlib.pyplot as plt


In [2]:
##Setting directory
dir_set='/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/001'
num_set=dir_set[-2:]
filenames=glob.glob(dir_set+'/*.tif')
##Creating lists and listing rededge band images 
filenamesIR=[]
filenamesREDGE=[]
for i in range(len(filenames)):
    if filenames[i][83]=='5':
        filenamesREDGE.append(filenames[i])
        
filenamesREDGE.sort()        
filenamesREDGE



['/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/001/IMG_0000_5.tif',
 '/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/001/IMG_0001_5.tif',
 '/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/001/IMG_0002_5.tif',
 '/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/001/IMG_0003_5.tif',
 '/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/001/IMG_0004_5.tif',
 '/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/001/IMG_0005_5.tif',
 '/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/001/IMG_0006_5.tif',
 '/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/001/IMG_0007_5.tif',
 '/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/001/IMG_0008_5.tif',
 '/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/001/IMG_0009_5.tif',
 '/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SE

In [3]:
##Listing near infrarred images
filenamesIR=[]

for i in range(len(filenames)):
    if filenames[i][83]=='4':
        filenamesIR.append(filenames[i])
        
filenamesIR.sort()         
filenamesIR

['/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/001/IMG_0000_4.tif',
 '/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/001/IMG_0001_4.tif',
 '/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/001/IMG_0002_4.tif',
 '/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/001/IMG_0003_4.tif',
 '/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/001/IMG_0004_4.tif',
 '/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/001/IMG_0005_4.tif',
 '/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/001/IMG_0006_4.tif',
 '/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/001/IMG_0007_4.tif',
 '/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/001/IMG_0008_4.tif',
 '/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/001/IMG_0009_4.tif',
 '/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SE

In [4]:
##Creating folder to save aligned images
if not os.path.exists(dir_set+'_IR-REDGE_ALIGNED/'):
    os.makedirs(dir_set+'_IR-REDGE_ALIGNED/')

In [5]:
##Preparing to align images
# Define motion model
warp_mode = cv2.MOTION_TRANSLATION  #(Options: MOTION_EUCLIDEAN, MOTION_AFFINE and MOTION_HOMOGRAPHY)
# Set the warp matrix to identity.
if warp_mode == cv2.MOTION_HOMOGRAPHY:
    warp_matrix = np.eye(3, 3, dtype=np.float32)
else:
    warp_matrix = np.eye(2, 3, dtype=np.float32)

In [6]:
# Set the stopping criteria for the algorithm.
criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 100000,  1e-5)

In [7]:
dir_ir_redge=dir_set+'_IR-REDGE_ALIGNED/'

print(dir_ir_redge)

/tidchile/collaborative/udd/ids/agro/AGRIFRUT_10-02/MICASENSE/0000SET/001_IR-REDGE_ALIGNED/


In [ ]:
##Aligning images and saving as .JPG

#Creating log file. List images which were not aligned

log = open(dir_ir_redge + "error_align.log", "w")

for i in range(len(filenamesREDGE)):
    try:
        print("Processing: " + filenamesREDGE[i][74:82])
      
        im_ir             =  cv2.imread(filenamesIR[i],0);
        im_redge          =  cv2.imread(filenamesREDGE[i],0);
    
        sz = im_redge.shape
        print (sz)
        height = int(sz[0])
        width = sz[1]
    
        im_aligned = np.zeros((height,width,3), dtype=np.uint8 )
        im_aligned[:,:,1] = im_redge[:,:]
    
        #Ir (Aligning Ir)
        
        (cc, warp_matrix) = cv2.findTransformECC(im_redge[:,:], im_ir[:,:], warp_matrix, warp_mode, criteria)
    
        im_aligned[:,:,0] = cv2.warpAffine(im_ir[:,:], warp_matrix, (width, height), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP);
        cv2.imwrite(dir_ir_redge + filenamesREDGE[i][74:82] + '_IRREDGE.JPG', im_aligned)
    except Exception: 
        print("Error processing: " + filenamesREDGE[i][74:82])
        log.write("Error processing: " + filenamesREDGE[i][74:82] + '\n')
        continue

In [ ]:
!ls

In [ ]:
##Copy information from red images (*_5.tif) with Exiftool. I aligning using as a reference the rededge band. 
#If you align using another band as a reference you have to use the images with 
#the correct extension.

In [ ]:
%%bash -s "$dir_set" "$dir_ir_redge" 

dir=$(pwd)

dir_set=$1/
dir_irr=$2

echo $dir_irr

ls $dir_irr*.JPG | xargs -n 1 basename > ${dir_irr}irr_files.txt

for f in `cat ${dir_irr}irr_files.txt`;
do 
aa=$dir_set${f:0:8}_5.tif
bb=$dir_irr$f

echo $aa
echo $bb
exiftool −overwrite_original_in_place -r -tagsFromFile $aa -gps:all -ExifIFD:all $bb; 
done

